In [46]:
import pandas as pd 
pd.set_option('max_columns', 100)
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 

In [47]:
train = pd.read_csv('happiness_train_complete.csv', encoding='ISO-8859-1')
test = pd.read_csv('happiness_test_complete.csv', encoding='ISO-8859-1')
IDtest = test['id']

### 合并训练集和测试机

In [48]:
train = train[train.happiness != -8]
train_len = len(train)

In [49]:
df = pd.concat(objs=[train,test], axis=0)

C:\Users\Voyager\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


In [62]:
df.head(2)

,birth,car,city,class,class_10_after,class_10_before,class_14,county,daughter,depression,edu,edu_other,edu_status,edu_yr,equity,f_birth,f_edu,f_political,f_work_14,family_income,family_m,family_status,floor_area,gender,happiness,health,health_problem,height_cm,house,hukou,hukou_loc,id,inc_ability,inc_exp,income,insur_1,insur_2,insur_3,insur_4,invest_0,invest_1,invest_2,invest_3,invest_4,invest_5,invest_6,invest_7,invest_8,invest_other,join_party,...,province,public_service_1,public_service_2,public_service_3,public_service_4,public_service_5,public_service_6,public_service_7,public_service_8,public_service_9,relax,religion,religion_freq,s_birth,s_edu,s_hukou,s_income,s_political,s_work_exper,s_work_status,s_work_type,socia_outing,social_friend,social_neighbor,socialize,son,status_3_before,status_peer,survey_time,survey_type,trust_1,trust_10,trust_11,trust_12,trust_13,trust_2,trust_3,trust_4,trust_5,trust_6,trust_7,trust_8,trust_9,view,weight_jin,work_exper,work_manage,work_status,work_type,work_yr
0,1959,2,32,3,3,3,1,59,0,5,11,NaN,4.0,-2.0,3,-2,4,4,1,60000.0,2,2,45.0,1,4.0,3,2,176,1,5,2.0,1,3,50000.0,20000,1,1,1,2,0,1,0,0,0,0,0,0,0,NaN,NaN,...,12,50,60.0,50,50,30.0,30,50,50,50,4,1,1,1958.0,6.0,5.0,40000.0,1.0,5.0,NaN,NaN,2,3.0,3.0,2,1,2,3,2015/8/4 14:18,1,4,3,-8,4,1,2,-8,-8,5,3,2,3,4,4,155,1,2.0,3.0,1.0,30.0
1,1992,2,52,6,8,4,5,85,0,3,12,NaN,4.0,2013.0,3,1972,3,1,2,40000.0,3,4,110.0,1,4.0,5,4,170,1,1,1.0,2,2,50000.0,20000,1,1,1,1,0,1,0,0,0,0,0,0,0,NaN,NaN,...,18,90,70.0,70,80,85.0,70,90,60,60,4,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2.0,6.0,2,0,1,1,2015/7/21 15:04,2,5,3,3,3,2,4,4,3,5,3,3,3,2,4,110,1,3.0,3.0,1.0,2.0


## 建模

In [89]:
train = df[:train_len]
test = df[train_len:]
# test.drop('city', axis=1, inplace=True)

In [128]:
Y_train = train['happiness'] 
# X_train = train.drop('happiness', axis=1)
predict = ['car', 'class', 'depression', 'gender', 'health', 'house']
X_train = train[predict]

In [129]:
df['class'].unique()

array([ 3,  6,  5,  1,  8,  2,  4,  7, 10, -8,  9], dtype=int64)

In [130]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV

kfold = StratifiedKFold(n_splits=5)
rf = RandomForestClassifier()
rf_params = {
    'max_features': ['auto', 'sqrt', 0.33],
    'n_estimators':[10]
}
gsrf = GridSearchCV(rf, param_grid=rf_params, cv=kfold, scoring='accuracy', n_jobs=-1, verbose=1)
gsrf.fit(X_train, Y_train)
gsrf.best_estimator_

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    5.5s finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=0.33, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [131]:
gsrf.best_score_

0.5945167751627441

In [171]:
test_happiness = pd.Series(gsrf.predict(test), name='happiness')
results = pd.concat([IDtest, test_happiness], axis=1)
results.to_csv('0830_submission.csv', index=False)

In [ ]:
pd.concat([IDtest, test_happiness], axis=1)